## Train and save the SVDTime Neural Network

The SVDTimeNN is a MultilayerPerceptron estimator of 
The truth data in this case are bin numbers in a series of time shift bins. The result of such a training is a distribution function for a time shift value. From these, it is easy to calculate mean value and standard deviation, but also do a range of approximate probabilistic calculations.

##### Required Python packages

The following python packages are used:
- math (basic python math functions)
- numpy (Vectors and matrices for numerics)
- pandas (Python analogue of Excel tables)
- matplotlib (Plotting library)
- seaborn (Advanced plotting)
- scipy (Scientific computing package)
- scikit-learn (machine learning)

Only sklear2pmml is missing in the current basf2 distribution. Install it with

pip3 install --user git+https://github.com/jpmml/sklearn2pmml.git

##### Other pre-requisites:

A sample of training data, plus binning and bounds information in pickle (*.pkl) files.

In [1]:
import math
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pylab
import seaborn as sns
from scipy import stats as stats
from scipy.optimize import minimize_scalar
from sklearn.neural_network import MLPClassifier
from sklearn2pmml import sklearn2pmml, PMMLPipeline
from svd.SVDSimBase import *
from lxml import etree as ET

### Retrieve training sample

In [2]:
n_samples = 1000000
pkl_name = 'SVDTime_Training{0}_{1}.pkl'

stockdata = pd.read_pickle(pkl_name.format('Sample', n_samples))
n_samples = len(stockdata)
bounds = pd.read_pickle(pkl_name.format('Bounds', n_samples))
bins = pd.read_pickle(pkl_name.format('Bins', n_samples))

timearray = bins['midpoint']
timebins = np.unique(bins[['lower','upper']])

In [3]:
stockdata.head()

,test,amplitude,t0,tau,sigma,s1,s2,s3,s4,s5,s6,normed_tau,t0_bin,abin
0,0.420250,45.085136,0.486506,271.287924,1.764844,0.000000,0.000000,30.597607,44.763166,42.496677,32.864097,44.632101,16,43
1,0.470006,40.923811,-76.319618,327.200027,1.863665,32.731206,40.779863,37.023823,32.194629,24.682549,18.780200,83.199794,1,38
2,0.752444,47.280162,18.665801,304.865337,2.584116,0.000000,0.000000,10.061467,38.310970,46.824519,43.728683,67.793515,20,45
3,0.700490,9.206401,-33.538889,230.937656,1.520102,0.000000,7.236357,8.552058,8.552058,6.578506,3.947104,16.798830,9,7
4,0.892159,54.405736,36.581858,344.323744,3.623510,0.000000,0.000000,0.000000,23.733895,48.571691,53.815226,95.011588,23,52


### Split the data into training and test samples

In [4]:
test_fraction = 0.2
X = stockdata[['s'+str(i) for i in range(1,7)]+['normed_tau']]
Y = stockdata['t0_bin']
X_train = X[stockdata.test>test_fraction]
X_test = X[stockdata.test<test_fraction]
Y_train = Y[stockdata.test>test_fraction]
Y_test = Y[stockdata.test<test_fraction]

In [5]:
X.head()

,s1,s2,s3,s4,s5,s6,normed_tau
0,0.000000,0.000000,30.597607,44.763166,42.496677,32.864097,44.632101
1,32.731206,40.779863,37.023823,32.194629,24.682549,18.780200,83.199794
2,0.000000,0.000000,10.061467,38.310970,46.824519,43.728683,67.793515
3,0.000000,7.236357,8.552058,8.552058,6.578506,3.947104,16.798830
4,0.000000,0.000000,0.000000,23.733895,48.571691,53.815226,95.011588


In [6]:
classifier = MLPClassifier(
    hidden_layer_sizes = (len(timearray)-1,len(timearray)+1),
    activation = 'relu',
    solver = 'adam',
    tol = 1.0e-6, 
    alpha = 0.005, 
    verbose = False
)
nntime_fitter = PMMLPipeline([('claasifier', classifier)])

In [7]:
nntime_fitter.fit(X_train,Y_train)

PMMLPipeline(steps=[('claasifier', MLPClassifier(activation='relu', alpha=0.005, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(24, 26), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False))])

In [8]:
test_score = nntime_fitter.score(X_test, Y_test)
train_score = nntime_fitter.score(X_train, Y_train)
print('Test: {}'.format(test_score))
print('Train: {}'.format(train_score))

Test: 0.7845886967015285
Train: 0.785154533652886


In [9]:
pmml_name = 'SVDTimeNet.pmml'
xml_name = pmml_name.replace('pmml','xml')
sklearn2pmml(nntime_fitter, pmml_name, with_repr = True)

In [10]:
parser = ET.XMLParser(remove_blank_text=True)
net = ET.parse(pmml_name, parser)
root = net.getroot()
# namespace hassle
namespace = root.nsmap[None]
nsprefix = '{'+namespace+'}'
procinfo = root.find(nsprefix + 'MiningBuildTask')

In [11]:
# Save some metadata
name = ET.SubElement(procinfo, nsprefix + 'Title')
name.text = 'Neural network for time shift estimation'

# Information on use of the classifier
target = ET.SubElement(procinfo, nsprefix + 'IntendedUse')
basf2proc = ET.SubElement(target, nsprefix + 'basf2process')
basf2simulation = ET.SubElement(basf2proc, nsprefix + 'Simulation')
basf2simulation.text = 'yes'
basf2reconstruction = ET.SubElement(basf2proc, nsprefix + 'Reconstruction')
basf2reconstruction.text = 'yes'
sensorType = ET.SubElement(target, nsprefix + 'SensorType')
sensorType.text = 'all'
sensorSide = ET.SubElement(target, nsprefix + 'SensorSide')
sensorSide.text = 'all'

#information on training
training = ET.SubElement(procinfo, nsprefix + 'Training')
source = ET.SubElement(training, nsprefix + 'SampleSource')
source.text = 'Toy simulation'
genfunc = ET.SubElement(training, nsprefix + 'Waveform')
genfunc.text = 'beta-prime'
num_samples = ET.SubElement(training, nsprefix + 'SampleSize')
train_samples = ET.SubElement(num_samples, nsprefix + 'Training', {'n': str(int((1-test_fraction)*n_samples))})
test_samples = ET.SubElement(num_samples, nsprefix + 'Test', {'n': str(int(test_fraction*n_samples))})
bounds.apply(
    lambda row: ET.SubElement(training, nsprefix + 'Parameter', **{u:str(v) for u, v in row.items()}), axis = 1
)

netparams = ET.SubElement(procinfo, nsprefix + 'NetworkParameters')
inputLayer = ET.SubElement(netparams, nsprefix + 'NetworkLayer')
inputLayer.attrib['number'] = str(0)
inputLayer.attrib['kind'] = 'input'
inputLayer.attrib['size'] = str(7) # 7 as in 6 APV samples + tau
n_hidden = len(classifier.hidden_layer_sizes)
for (iLayer, sz) in zip(range(1,1+n_hidden), classifier.hidden_layer_sizes) :
    layer = ET.SubElement(netparams, nsprefix + 'NetworkLayer')
    layer.attrib['number'] = str(iLayer)
    layer.attrib['kind'] = 'hidden'
    layer.attrib['size'] = str(sz)
outputLayer = ET.SubElement(netparams, nsprefix + 'NetworkLayer')
outputLayer.attrib['number'] = str(n_hidden+1)
outputLayer.attrib['kind'] = 'output'
outputLayer.attrib['size'] = str(len(timearray)) 

for field in root.find(nsprefix + 'DataDictionary'):
    if field.attrib['name'] == 't0_bin':
        for child in field:
            i = int(child.attrib['value'])
            child.attrib['lower'] = '{0:.3f}'.format(bins.loc[i,'lower'])
            child.attrib['upper'] = '{0:.3f}'.format(bins.loc[i,'upper'])
            child.attrib['midpoint'] = '{0:.3f}'.format(bins.loc[i,'midpoint'])


In [12]:
net.write(xml_name, xml_declaration = True, pretty_print = True, encoding = 'utf-8')

#### Set up tau en/decoder

In [13]:
amp_index = bounds[bounds.value == 'amplitude'].index[0]
amp_range = (bounds.ix[amp_index,'low'], bounds.ix[amp_index, 'high'])
tau_index = bounds[bounds.value == 'tau'].index[0]
tau_range = (bounds.ix[tau_index,'low'], bounds.ix[tau_index, 'high'])
coder = tau_encoder(amp_range, tau_range)

/home/kvasnicka/work/belle2_new/externals/v01-05-01/Linux_x86_64/common/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


#### True values

In [14]:
Trues_test = stockdata[stockdata.test < test_fraction][['t0', 'amplitude','tau','t0_bin', 'abin']]

In [15]:
Trues_test.head()

,t0,amplitude,tau,t0_bin,abin
9,-68.156099,5.824593,268.574949,3,3
19,18.888240,60.214912,277.003488,20,58
22,19.213061,71.931029,341.617938,20,69
28,-3.668041,46.327342,287.331662,15,44
32,4.855365,36.319822,224.405947,17,34


#### Predicted probabilities.

In [16]:
probs = nntime_fitter.predict_proba(X_test)
probs

array([[  5.86533709e-01,   3.02595348e-01,   8.87765392e-02, ...,
          1.90258978e-05,   6.28298443e-05,   1.77978476e-04],
       [  1.20169954e-34,   2.58969988e-29,   7.14542124e-27, ...,
          2.91081205e-14,   7.84909697e-27,   7.56510029e-42],
       [  2.34807906e-30,   1.06363566e-24,   3.43091128e-22, ...,
          1.05000188e-12,   8.70708634e-26,   2.04175979e-41],
       ..., 
       [  7.32980851e-01,   2.63783246e-01,   3.23361570e-03, ...,
          8.35669447e-17,   5.73346809e-18,   4.38920101e-21],
       [  2.15059669e-43,   8.45856817e-34,   6.14335961e-27, ...,
          8.24455997e-01,   1.68920400e-05,   1.84031479e-14],
       [  1.04366005e-26,   9.01535175e-22,   3.63215696e-18, ...,
          8.62234726e-02,   1.25711217e-04,   5.07510715e-09]])

### Calculate time shifts and amplitudes from probabilities

In [17]:
def fitFromProb(fw, signals, p, tau, timearray):
    t_fit = np.average(timearray, weights = p)
    t_sigma = np.sqrt(np.average((timearray - t_fit)**2, weights = p))
    weights = fw(-t_fit + np.linspace(-dt, 4*dt, 6, endpoint = True), tau = tau)
    weights[signals.values == 0.0] = 0.0
    norm = 1.0 / np.inner(weights, weights)
    a_fit = np.inner(signals, weights) * norm
    a_sigma = np.sqrt(norm)
    residuals = signals - a_fit * weights
    ndf = np.sum(np.ones_like(signals[signals>0])) - 2 # Can't be less than 1
    chi2_ndf = np.inner(residuals, residuals)/ndf
    return pd.Series({
        't_fit':t_fit, 
        't_sigma':t_sigma, 
        'a_fit':a_fit, 
        'a_sigma':a_sigma,
        'chi2_ndf':chi2_ndf
            })

In [18]:
probdf = pd.DataFrame(probs)
probdf.index = X_test.index
probdf.to_pickle('SVDTime_TrainingProbs_{0}.pkl'.format(n_samples))

In [19]:
fits = X_test.apply(
    lambda row: fitFromProb(
        betaprime_wave, 
        row[['s'+str(i) for i in range(1,7)]], 
        probdf.ix[row.name],
        coder.decode(row['normed_tau']), 
        timearray), 
    axis = 1
)
fits['t_true'] = Trues_test['t0']
fits['tau'] = Trues_test['tau']
fits['a_true'] = Trues_test['amplitude']
fits['t_bin'] = Trues_test['t0_bin']
fits['a_bin'] = Trues_test['abin']

/home/kvasnicka/work/belle2_new/externals/v01-05-01/Linux_x86_64/common/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [20]:
fits.head()

,a_fit,a_sigma,chi2_ndf,t_fit,t_sigma,t_true,tau,a_true,t_bin,a_bin
9,5.863042,0.629765,2.177858,-73.264863,4.286311,-68.156099,268.574949,5.824593,3,3
19,60.271050,0.634976,2.387612,19.388081,0.567940,18.888240,277.003488,60.214912,20,58
22,72.868984,0.639596,2.091510,19.386944,0.595397,19.213061,341.617938,71.931029,20,69
28,45.220982,0.591613,0.444565,-2.385301,2.408197,-3.668041,287.331662,46.327342,15,44
32,36.399106,0.634107,0.809875,4.663393,1.201728,4.855365,224.405947,36.319822,17,34


In [21]:
fits.to_pickle('SVDTime_TrainingFits_{0}.pkl'.format(n_samples))

In [22]:
import pickle

In [23]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)


In [24]:
with open('classifier.txt', 'w') as cdump:
    cdump.write("Classifier coefficients:\n")
    for iLayer in range(len(classifier.coefs_)):
        cdump.write('Layer: {0}\n'.format(iLayer))
        nrows = classifier.coefs_[iLayer].shape[0]
        ncols = classifier.coefs_[iLayer].shape[1]
        cdump.write('Weights:\n')
        for col in range(ncols):
            s = " ".join([str(classifier.coefs_[iLayer][row, col]) for row in range(nrows)])
            s+="\n"
            cdump.write(s)
        # intercepts should have nrows dimension
        cdump.write('Intercepts:\n')
        s = " ".join([str(classifier.intercepts_[iLayer][col]) for col in range(ncols)])
        s += "\n"
        cdump.write(s)

print("Data written.")

Data written.
